In [1]:
import pandas as pd
import numpy as np
import pickle
from LightGBMwithBayesOpt import LightGBMwithBayesOpt, LightGBMEvalutor

In [2]:
### data for 0 plan 
### all three category columns of channel, fee_prod_id and mbr_card_gr are already label-encoded
### all numeric columns are already standardization-scaled

with open('train_set_0304.pkl', 'rb') as f:
    train_X, train_y = pickle.load(f)
with open('test_set_0405.pkl', 'rb') as f:
    test_X, test_y = pickle.load(f)

In [3]:
train_X.head()

,channel,fee_prod_id,mbr_card_gr,additional_svc_allcare_scrb_type_standard,additional_svc_ansim_option_scrb_type_standard,additional_svc_bugs_scrb_type_standard,additional_svc_flo_scrb_type_standard,additional_svc_melon_scrb_type_standard,additional_svc_oksusu_scrb_type_standard,additional_svc_pooq_scrb_type_standard,...,overage_bf0_standard,overage_bf1_standard,overage_bf2_standard,real_avg_arpu_standard,rem_allot_cnt_standard,roaming_cnt_m3_standard,simpl_ref_cnsl_cnt_m3_standard,smart_watch_cnt_standard,svc_scrb_period_standard,tablet_cnt_standard
252483,1,247,2,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,2.317187,0.0,...,-0.202224,-0.191739,-0.201029,-0.232049,-0.920419,-0.117019,-0.338241,-0.057639,3.161220,-0.031772
252484,2,171,2,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-0.027538,-0.498600,-0.117019,0.233286,-0.057639,2.691778,-0.031772
252485,3,204,4,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,0.198199,-0.393145,-0.117019,-0.338241,-0.057639,2.626016,-0.031772
252486,2,168,4,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-0.587029,1.399585,-0.117019,0.233286,-0.057639,2.314404,-0.031772
252487,2,141,0,-0.019464,-0.290523,-0.114694,-0.11104,-0.075765,0.528471,0.0,...,-0.202224,-0.191739,-0.201029,-1.172890,-0.920419,-0.117019,1.376341,-0.057639,2.307322,-0.031772


In [4]:
pd.value_counts(train_y)

0Large      11222
Special      5237
0Medium      4969
Essence      3326
0Small       3275
Safe2.5G     1242
Safe4G        419
Max           419
Name: price_plan, dtype: int64

In [5]:
train_y = list(map(lambda x: int(x in ['0Small', '0Medium', '0Large']), train_y))
test_y = list(map(lambda x: int(x in ['0Small', '0Medium', '0Large']), test_y))

In [6]:
pd.value_counts(train_y)

0    34401
1    19466
dtype: int64

In [7]:
not_range_params = {
    'boosting': ['gbdt', 'dart'],
    'objective': ['binary'],
    'is_unbalance': [False, True]
}
rangeable_params = {
    'num_leaves': (20, 100),
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.5),
    'num_iterations': (100, 300),
    'bagging_fraction': (0.8, 1),
    'bagging_freq': (2, 10),
    'feature_fraction': (0.6, 1)
}
int_params = ['num_leaves', 'max_depth', 'num_iterations', 'bagging_freq']

In [8]:
#### binary-classification modeling for mobility to target price plans or not
#### while taking the category columns as numeric
#### 
binary_tuner = LightGBMwithBayesOpt(train_X, train_y, 3, rangeable_params, int_params, 5, [0, 1], not_range_params,  
                                    pred_modes=['cut_off'], metrics=['accuracy', 'mcc'], thresholds=[0.5])
binary_tuner.do_parameter_tuning()

|   iter    |  target   | baggin... | baggin... | boosting  | featur... | is_unb... | learni... | max_depth | num_it... | num_le... | objective |
-------------------------------------------------------------------------------------------------------------------------------------------------
{'bagging_fraction': 0.932022301181832, 'bagging_freq': 9, 'boosting': 'dart', 'feature_fraction': 0.6893783798723037, 'is_unbalance': False, 'learning_rate': 0.4736757344908684, 'max_depth': 8, 'num_iterations': 252, 'num_leaves': 71, 'objective': 'binary', 'num_threads': 20}
0


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


   accuracy_cut_off  mcc_cut_off
0          0.720985     0.384495
1          0.718089     0.379124
2          0.716625     0.374914
mean: 0.5490385428055848
|  1        |  0.549    |  0.932    |  9.347    |  0.5782   |  0.6894   |  0.3067   |  0.4737   |  8.336    |  252.5    |  70.72    |  0.0      |
{'bagging_fraction': 0.904662875282128, 'bagging_freq': 3, 'boosting': 'dart', 'feature_fraction': 0.9914984962352847, 'is_unbalance': False, 'learning_rate': 0.41572966614999024, 'max_depth': 7, 'num_iterations': 124, 'num_leaves': 35, 'objective': 'binary', 'num_threads': 20}
0


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


   accuracy_cut_off  mcc_cut_off
0          0.721040     0.381373
1          0.726275     0.395789
2          0.727207     0.397035
mean: 0.5581199138942982
|  2        |  0.5581   |  0.9047   |  3.076    |  0.8907   |  0.9915   |  0.09683  |  0.4157   |  6.844    |  123.7    |  34.87    |  0.0      |
{'bagging_fraction': 0.9289534733913996, 'bagging_freq': 4, 'boosting': 'gbdt', 'feature_fraction': 0.7949614570842739, 'is_unbalance': True, 'learning_rate': 0.11805762201325287, 'max_depth': 10, 'num_iterations': 221, 'num_leaves': 44, 'objective': 'binary', 'num_threads': 20}
0


...

   accuracy_cut_off  mcc_cut_off
0          0.713912     0.349961
1          0.721374     0.369083
2          0.721693     0.369225
mean: 0.5408747113485463
|  13       |  0.5409   |  0.9781   |  2.0      |  0.1443   |  1.0      |  0.0      |  0.01     |  10.0     |  208.6    |  100.0    |  0.0      |
{'bagging_fraction': 0.9682024826323868, 'bagging_freq': 10, 'boosting': 'gbdt', 'feature_fraction': 1.0, 'is_unbalance': False, 'learning_rate': 0.01, 'max_depth': 3, 'num_iterations': 245, 'num_leaves': 20, 'objective': 'binary', 'num_threads': 20}
0


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


   accuracy_cut_off  mcc_cut_off
0          0.674204     0.231223
1          0.687458     0.267009
2          0.681704     0.249555
mean: 0.46519230543806606
|  14       |  0.4652   |  0.9682   |  10.0     |  3.131e-0 |  1.0      |  0.3547   |  0.01     |  3.0      |  245.0    |  20.0     |  0.0      |
{'bagging_fraction': 0.8, 'bagging_freq': 2, 'boosting': 'gbdt', 'feature_fraction': 0.6, 'is_unbalance': False, 'learning_rate': 0.3545618875908192, 'max_depth': 3, 'num_iterations': 300, 'num_leaves': 54, 'objective': 'binary', 'num_threads': 20}
0


/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


   accuracy_cut_off  mcc_cut_off
0          0.717086     0.375001
1          0.721709     0.384214
2          0.723865     0.391438
mean: 0.5522187346091937
|  15       |  0.5522   |  0.8      |  2.0      |  0.0      |  0.6      |  0.0      |  0.3546   |  3.0      |  300.0    |  54.45    |  0.0      |


In [9]:
### train data with the best parameter
binary_model = binary_tuner.do_train_with_given_parameter(binary_tuner.best_parameter, binary_tuner.data_set)

/home/dmig/py_libs/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [10]:
binary_model.params

{'bagging_fraction': 0.8868101711580003,
 'bagging_freq': 8,
 'boosting': 'gbdt',
 'feature_fraction': 0.7304283525982124,
 'is_unbalance': True,
 'learning_rate': 0.10147903653166253,
 'max_depth': 8,
 'num_leaves': 39,
 'num_threads': 20,
 'objective': 'binary'}

In [12]:
### evaluate the model with the test set
evl = LightGBMEvalutor(binary_model, test_X, [0, 1], test_y)
scores = evl.score_predict()
labels = evl.label_predict(scores, mode='cut_off', threshold=0.5)
acc_mcc_list = list(map(lambda x: evl.measure_pred_with_true(labels, mode=x), ['accuracy', 'mcc']))

In [14]:
scores[:10]

array([0.4126598 , 0.34422864, 0.13353345, 0.45986849, 0.44819393,
       0.6031457 , 0.45839246, 0.27097308, 0.40673216, 0.37362474])

In [15]:
labels[:10]

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

In [16]:
## accuracy, mcc
acc_mcc_list

[0.6213629402756509, 0.19362433367049303]